In [1]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt 
import time

from collections import Counter
from collections import defaultdict
from wordcloud import WordCloud

## Read Crunchbase Data

In [2]:
df = pd.read_csv('crunchbase_data/crunchbase_blockchain_companies_raw.csv', encoding='utf-8')
df['Organization Name'] = df['Organization Name'].str.strip()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50) # Show full text
df

,Organization Name,Categories,Headquarters Location,Description,CB Rank (Company),Founded Date,Investor Type,Headquarters Regions,Estimated Revenue Range,Operating Status,Exit Date,Closed Date,Company Type,Website,Full Description,Investment Stage,Number of Portfolio Organizations,Number of Investments,Number of Lead Investments,Number of Exits,Number of Exits (IPO),Accelerator Program Type,Accelerator Application Deadline,Accelerator Duration (in weeks),School Type,School Program,Number of Enrollments,School Method,Number of Founders (Alumni),Number of Alumni,Category Groups,Number of Founders,Founders,Number of Employees,Number of Funding Rounds,Funding Status,Last Funding Date,Last Funding Amount,Last Funding Type,Last Equity Funding Amount,Last Equity Funding Type,Total Equity Funding Amount,Total Funding Amount,Top 5 Investors,Number of Lead Investors,Number of Investors,Acquisition Status,Number of Acquisitions,Transaction Name,Acquired by,Announced Date,Price,Acquisition Type,Acquisition Terms,IPO Status,IPO Date,Delisted Date,Money Raised at IPO,Valuation at IPO,Stock Symbol,Stock Exchange,Number of Events,CB Rank (Organization),CB Rank (School),Trend Score (7 Days),Trend Score (30 Days),Trend Score (90 Days),Monthly Visits,Average Visits (6 months),Monthly Visits Growth,Visit Duration,Visit Duration Growth,Page Views / Visit,Page Views / Visit Growth,Bounce Rate,Bounce Rate Growth,Global Traffic Rank,Monthly Rank Change (#),Monthly Rank Growth,Active Tech Count,Number of Apps,Downloads Last 30 Days,Total Products Active,Patents Granted,Trademarks Registered,Most Popular Patent Class,Most Popular Trademark Class,IT Spend,Most Recent Valuation Range,Date of Most Recent Valuation,Contact Email,Phone Number,Number of Articles,Hub Tags
0,Kraken,"Asset Management, Blockchain, Cryptocurrency,...","San Francisco, California, United States",Kraken is a cryptocurrency exchange that provi...,49,"Jul 28, 2011",—\n,"San Francisco Bay Area, West Coast, Western US",$50M to $100M,Active,—,—,For Profit,www.kraken.com/,Our mission is to accelerate the adoption of c...,—\n,—,—,—,—,—,—,—,—,—,—,—,—,—,—,"Financial Services, Lending and Investments, ...",1,Jesse Powell,501-1000,4,—,"May 21, 2019","$13,500,000",Equity Crowdfunding,"$13,500,000",Equity Crowdfunding,"$118,500,000","$118,500,000","Blockchain Capital, BnkToTheFuture, Digital C...",1,6,Made Acquisitions\n,10,—,—,—,—,—,—,Private,—,—,—,—,—,—,2,59,—,-2.0,-0.1,9.0,"2,737,958","3,119,818.28",-3.16%,539,-5.4%,6.66,-4.78%,37.37%,3.08%,"18,642","1,514",8.84%,—,—,—,19,0,2,—,Insurance; financial affairs,"$163,179",—,—,support@kraken.com,—,44,Unicorn\n
1,Securitize,"Blockchain, Cryptocurrency, Finance, Financia...","San Francisco, California, United States",We deliver trusted global solutions for creati...,257,"Jan 18, 2018",—\n,"San Francisco Bay Area, West Coast, Western US",Less than $1M,Active,—,—,For Profit,securitize.io/,Securitize is a primary issuance platform and ...,—\n,—,—,—,—,—,—,—,—,—,—,—,—,—,—,"Financial Services, Information Technology, P...",4,"Carlos Domingo, Jamie H. Finn, Shay Finkelste...",11-50,5,Early Stage Venture,"Feb 5, 2020",—,Series A,—,Series A,"$26,750,000","$26,750,000","Ripple, Blockchain Capital, Global Brain Corp...",2,24,Made Acquisitions\n,1,—,—,—,—,—,—,Private,—,—,—,—,—,—,1,277,—,-0.9,1.1,8.9,"7,191","7,012.34",5.05%,"1,531",9.43%,5.26,-7.37%,51.58%,15.99%,"1,775,410","-61,315",-3.34%,68,—,—,—,0,3,—,Scientific and technological services,—,—,—,info@securitize.io,+1 415-413-8993,15,—\n
2,REBEL,"Blockchain, Consumer Lending, Credit, FinTech...","São Paulo, Sao Paulo, Brazil","Alternative Online Lending, FinTech, online cr...",321,2015,—\n,Latin America,$1M to $10M,Active,—,—,For Profit,www.rebel.com.br,"REBEL is a data, tech and analytics-driven onl...",—\n,—,—,—,—,—,—,—,—,—,—,—,—,—,—,"Financial Services, Lending and Investments",3,"Andre Botelho Bastos, Paulo Asterio Nunes, Ra...",11-50,8,Early Stage Venture,"Dec 3, 2019","$10,000,000",Series B,"$10,000,000

### Validate the dataset

In [3]:
# Note: about 100 points were omitted due to API call results being inconsistent. Check that the data has important companies:
names_in_df = [name.lower().strip() for name in df['Organization Name'].tolist()]

names_to_find = ['cryptokitties', 'consensys', 'pixelplex', 'limechain']
for name in names_to_find:
    if name in names_in_df:
        print(name + ' found!')
    else:
        print(name + ' not found...')

# Check startups on the Master List
masterlist_startups = None
with open('masterlist_data/blockchain_startup_names.txt', 'r') as f:
    masterlist_startups = [line.lower().strip() for line in f]
    
found = []
for name in names_in_df:
    if name in masterlist_startups:
        found.append(name)
        
print('\n%d out of %d companies found in master list:' % (len(found), len(masterlist_startups)))
print(found)

cryptokitties found!
consensys not found...
pixelplex not found...
limechain found!

42 out of 111 companies found in master list:
['ripple', 'circle', 'coinbase', 'veem', 'cashaa', 'propy', 'trusttoken', 'shipchain', 'latoken', 'parity technologies', 'provenance', 'omisego', 'bloq', 'bigchaindb', 'sensay', 'decent', 'hive blockchain', 'aeternity', 'dfinity', 'lisk', 'humaniq', 'crowdz', 'minthealth', 'hyperledger', 'connectjob', 'horizon state', 'epiphyte', 'vechain', 'colony', 'monax', 'icobox', 'global blockchain', 'cypherium', 'luna', 'blockmedx', 'bitse', 'chain of things', 'herosphere', 'decent', 'openchain', 'backfeed', 'decent']


### Keep only the relevant columns

In [4]:
# Filter columns
df = df[['Organization Name','Categories', 'Description', 'Founded Date', 'Estimated Revenue Range', 'Operating Status', 'Exit Date', 'Closed Date', 'Full Description', 'Category Groups', 'Website']]
df

,Organization Name,Categories,Description,Founded Date,Estimated Revenue Range,Operating Status,Exit Date,Closed Date,Full Description,Category Groups,Website
0,Kraken,"Asset Management, Blockchain, Cryptocurrency,...",Kraken is a cryptocurrency exchange that provi...,"Jul 28, 2011",$50M to $100M,Active,—,—,Our mission is to accelerate the adoption of c...,"Financial Services, Lending and Investments, ...",www.kraken.com/
1,Securitize,"Blockchain, Cryptocurrency, Finance, Financia...",We deliver trusted global solutions for creati...,"Jan 18, 2018",Less than $1M,Active,—,—,Securitize is a primary issuance platform and ...,"Financial Services, Information Technology, P...",securitize.io/
2,REBEL,"Blockchain, Consumer Lending, Credit, FinTech...","Alternative Online Lending, FinTech, online cr...",2015,$1M to $10M,Active,—,—,"REBEL is a data, tech and analytics-driven onl...","Financial Services, Lending and Investments",www.rebel.com.br
3,Viberate,"Analytics, Big Data, Blockchain, Information ...",As the worlds largest resource of reliable inf...,"Dec 9, 2015",$1M to $10M,Active,—,—,Viberate is a music network that collects and ...,"Data and Analytics, Design, Information Techn...",www.viberate.com/
4,Ripple,"Blockchain, Cryptocurrency, Financial Service...",Ripple provides one frictionless experience to...,2012,$10M to $50M,Active,—,—,Ripple provides one frictionless experience to...,"Financial Services, Internet Services, Paymen...",ripple.com
...,...,...,...,...,...,...,...,...,...,...,...
5298,Copycash,"Blockchain, Cryptocurrency",Copycash is a blockchain-based platform connec...,—,—,Active,—,—,—,"Financial Services, Payments, Software",www.copycash.io/
5299,XinFin,"Blockchain, Internet",XinFin is an open source hybrid blockchain pla...,"Jan 1, 2017",—,Active,—,—,—,Internet Services,www.xinfin.org/
5300,Mining Rig Club,"Blockchain, Internet",Mining Rig Club provides an array of products ...,"Jan 1, 2017",—,Active,—,—,—,Internet Services,www.miningrigclub.com/
5301,Ineeza,"Blockchain, Information Services",Ineeza is a technology company that supports e...,"Oct 5, 2015",—,Active,—,—,—,Information Technology,www.ineeza.com/


# Data Exploration

### See Datapoints based on Founded Year

In [5]:
# Create separate dfs by "Founded Year"
pd.options.mode.chained_assignment = None  # default='warn'
founded_years = []
for i in range(len(df)):
    year = df.loc[i, 'Founded Date'].split(' ')[-1].strip()
    if year != '—':
        assert(year[:2] == '20' or year[:2] == '19' or year[:2] == '18')
        assert(len(year) == 4)
        founded_years.append(year)
    else:
        founded_years.append(None)
        
df['Founded Year'] = founded_years

dfs_by_year = []
unique_years = sorted(list(set(filter(None, founded_years))))

# Only count companies founded after 2008 (blockchain startups not possible before then)
unique_years = list(filter(lambda year: int(year) >= 2008, unique_years))

for year in unique_years:
    year_df = df[df['Founded Year'] == year]
    
    dfs_by_year.append(year_df)
    
#df = df.drop(columns=['Founded Year'])
    
print('%d years found in data are >= 2008' % len(dfs_by_year))
print('Number of companies in each year:')
for d in dfs_by_year:
    print('%d ' % len(d), end='')

13 years found in data are >= 2008
Number of companies in each year:
23 34 48 56 66 146 225 276 507 1478 1261 192 5 

In [6]:
pd.set_option('display.max_colwidth', None) # Show full text
dfs_by_year[0]

,Organization Name,Categories,Description,Founded Date,Estimated Revenue Range,Operating Status,Exit Date,Closed Date,Full Description,Category Groups,Website,Founded Year
644,CloseCross,"Blockchain, Internet, Software",CloseCross is the first fully decentralized derivatives platform.,2008,—,Active,—,—,CloseCross opens up the massive financial derivatives market to the public giving them easy access to peer-to-peer derivative trading platform without any need for deep financial knowledge.,"Internet Services, Software",closecross.com/,2008
746,InXero,"Blockchain, Digital Marketing, Information Technology, Marketing, SaaS, Sales Automation, Social Media Marketing",InXero is industry’s first Sales Enablement and Go-to-Market platform for Solution Providers,2008,—,Active,—,—,"75% of world trade runs through solution providers / resellers. Yet taking new products and solutions to market remains massively complicated, highly manual, and risk-prone. There has been no simple, efficient and integrated way for resellers to market and sell their solutions. Until now.\n\nAt InXero, our mission is to simplify the entire go-to-market process so that resellers can move fast and sell solutions effortlessly - all with the latest industry intelligence, vendor content and digital reach. That’s why we have delivered the InXero platform - the industry’s first Sales Enablement and Go-to-Market platform for marketing and selling of single- and multi-vendor solutions for unique buyer needs. With InXero, your team will have all the benefits of close collaboration with your buyer, vendor and distributor partners while gaining the speed and agility needed to win in this fast-paced, dynamic world.","Information Technology, Sales and Marketing, Software",www.inxero.com,2008
1231,SIX Swiss Exchange,"Blockchain, Payments",SIX Swiss Exchange is a stock exchange company.,2008,$10M to $50M,Active,—,—,—,"Financial Services, Payments",www.six-swiss-exchange.com/index.html,2008
1250,Bitcoin,"Bitcoin, Blockchain, Cryptocurrency, Finance, Mining, Payments, Virtual Currency",Bitcoin.com is the premier source for everything Bitcoin related.,2008,—,Active,—,—,"Bitcoin.com is your premier source for everything Bitcoin related. They can help you buy Bitcoin and open a secure Bitcoin wallet. You can also read the latest news or engage with the community on the Bitcoin Forum. Please keep in mind that this is a commercial website that lists wallets, exchanges and other Bitcoin related companies and information.","Financial Services, Natural Resources, Payments, Software",bitcoin.com,2008
1340,ArcTouch,"Android, Apps, Augmented Reality, Blockchain, Digital Marketing, Internet of Things, iOS, Smart Home, Software",We design lovable apps and develop technology for the connected world.,2008,—,Active,"Jan 27, 2016",—,"We're the app developers and designers behind well known products from hundreds of Fortune 500 companies, world-class brands, and influential startups. We specialize in app design and app development for iOS, Android, and Xamarin. We also help companies with their Alexa skills, web sites, back-end APIs, blockchain development, augmented reality experiences, and IoT smart products.","Apps, Consumer Electronics, Hardware, Internet Services, Mobile, Platforms, Real Estate, Sales and Marketing, Software",arctouch.com,2008
1354,LivaRava,"Blockchain, Knowledge Management, PaaS",Probably is the best tool for knowledge management and sharing.,2008,Less than $1M,Active,—,—,"LivaRava is a web-based knowledge management system that allows its users to systematize and organize information such as text, images, audios, videos, and others. It also enables its users to share information with their friends.\n\nLivaRava was launched in 2008 and is operated from Nevada, United States.",Software,www.livarava.com,2008
1556,Digital Vega,"Banking, Blockchain, Financial Services, Marketplace, Training",Digital Vega is a privately held company operating the Medusa FX Option trading platf

### Create histograms for different years

In [206]:
def get_column_words(df, column_name, year=None):
    if year:
        df = df[df['Founded Year'] == year]
    words_list = []
    for words in df[column_name].tolist():
        words = list(map(str.strip, words.split(',')))
        words = list(filter(lambda x: x != '—', words))
        words_list.extend(words)
    return words_list

def get_column_words_unique(df, column_name, year=None):
    words_list = get_column_words(df, column_name, year=None)
    unique_words_list = sorted(list(set(words_list)))
    return unique_words_list

def list_to_dict(list_item, vocab=None):
    if vocab:
        words_count_dict = dict.fromkeys(vocab,0)
        for word in list_item:
            words_count_dict[word] += 1
        return words_count_dict
    return Counter(list_item)

def show_word_cloud(words_list, title=''):
    words_count_dict = list_to_dict(words_list)
    wordcloud = WordCloud(width = 800, height = 800, 
                    background_color ='white',
                    min_font_size = 10)
    wordcloud.generate_from_frequencies(words_count_dict)
                    
    plt.figure(figsize = (3, 3), facecolor = None)
    plt.imshow(wordcloud) 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    if title != '':
        plt.title(title)
    plt.show()
    
def create_histogram(words_list, vocab, title='', savepath='', max_y=None, show=True):
    words_count_dict = get_hist_data(words_list, vocab)
        
    plt.ioff()
    fig = plt.figure(figsize = (10, 10), facecolor = None);
    if title != '':
        plt.title(title);
    plt.bar(list(words_count_dict.keys()), words_count_dict.values(), color='g');
    plt.xticks(rotation='vertical');
    if max_y:
        plt.ylim((0, max_y));
    plt.tight_layout();
    if savepath != '':
        plt.savefig(savepath);
        plt.close(fig)
        print('histogram saved: %s' % savepath)
    if show:
        plt.show()
        
    
def get_hist_data(words_list, vocab):
    words_count_dict = list_to_dict(words_list, vocab)
    words_count_dict = dict(sorted(words_count_dict.items(), key=lambda x: x[0]))
    return words_count_dict

In [207]:
hist_data = []

categories_vocab = get_column_words_unique(df, 'Category Groups')
for year in unique_years:
    categories = get_column_words(df, 'Category Groups', year=year)
    create_histogram(categories, categories_vocab, title=year, max_y=700, show=False, savepath='images/Category_Groups_%s.png' % year)

histogram saved: images/Category_Groups_2008.png
histogram saved: images/Category_Groups_2009.png
histogram saved: images/Category_Groups_2010.png
histogram saved: images/Category_Groups_2011.png
histogram saved: images/Category_Groups_2012.png
histogram saved: images/Category_Groups_2013.png
histogram saved: images/Category_Groups_2014.png
histogram saved: images/Category_Groups_2015.png
histogram saved: images/Category_Groups_2016.png
histogram saved: images/Category_Groups_2017.png
histogram saved: images/Category_Groups_2018.png
histogram saved: images/Category_Groups_2019.png
histogram saved: images/Category_Groups_2020.png
